<a href="https://colab.research.google.com/github/Mabinogit/AI-Image-Classification/blob/main/Bank_Renege.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This example models a bank counter and customers arriving at random times. Each customer has a certain patience. She waits to get to the counter until she’s at the end of her tether. If she gets to the counter, she uses it for a while before releasing it.

In [1]:
!pip install simpy

import random
import simpy

In [2]:

random_seed = 42
NEW_CUSTOMERS = random.randint(25,50)  # Total number of customers
INTERVAL_CUSTOMERS = random.randint(1,25)  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience
cus_serviced = 0
cus_renaged = 0

In [3]:
# Setup and start the simulation
print('Bank renege')
random.seed(random_seed)
env = simpy.Environment()
counter = simpy.Resource(env, capacity=2)
result_store = simpy.Store(env)  # Create a Store to hold results

Bank renege


In [4]:
def source(env, number, interval, counter,  result_store ):
    """Source generates customers randomly"""
    for i in range(number):
      c = customer(env, f'Customer{i:02d}', counter, time_in_bank=12.0,  result_store=result_store )
      env.process(c)
      t = random.expovariate(1.0 / interval)
      yield env.timeout(t)


In [7]:
def customer(env, name, counter, time_in_bank,  result_store ):
    """Customer arrives, is served and leaves."""

    arrive = env.now
    print(f'{arrive:7.4f} {name}: Here I am')


    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(patience)

        wait = env.now - arrive

        if req in results:
            # We got to the counter
            print(f'{env.now:7.4f} {name}: Waited {wait:6.3f}')
            yield result_store.put('serviced')  # Put 'serviced' into the store

            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print(f'{env.now:7.4f} {name}: Finished')



        else:
            # We reneged
            print(f'{env.now:7.4f} {name}: RENEGED after {wait:6.3f}')
            yield result_store.put('reneged')  # Put 'reneged' into the store


 random.uniform - used to generate a random floating-point number within a specified range.

 random.expovariate - generates a random number following an exponential distribution. Often used to model the time between events, where events occur continuously and independently at a constant average rate.

In [8]:
# Start processes and run

env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter, result_store))
env.run()


print(f'Serviced: {cus_serviced}, Reneged: {cus_renaged}')

646.0130 Customer00: Here I am
646.0130 Customer00: Waited  0.000
646.9782 Customer01: Here I am
646.9782 Customer01: Waited  0.000
647.7658 Customer01: Finished
657.8614 Customer00: Finished
678.3902 Customer02: Here I am
678.3902 Customer02: Waited  0.000
687.4279 Customer02: Finished
688.0032 Customer03: Here I am
688.0032 Customer03: Waited  0.000
688.1418 Customer03: Finished
758.8665 Customer04: Here I am
758.8665 Customer04: Waited  0.000
768.1061 Customer04: Finished
784.3775 Customer05: Here I am
784.3775 Customer05: Waited  0.000
785.7968 Customer05: Finished
790.5849 Customer06: Here I am
790.5849 Customer06: Waited  0.000
801.9952 Customer07: Here I am
801.9952 Customer07: Waited  0.000
810.3270 Customer07: Finished
827.4863 Customer06: Finished
843.7209 Customer08: Here I am
843.7209 Customer08: Waited  0.000
847.6571 Customer09: Here I am
847.6571 Customer09: Waited  0.000
854.7462 Customer10: Here I am
857.2712 Customer10: RENEGED after  2.525
858.0635 Customer11: Here I